# Manufacturing Test Data Analysis

This jupyter notebook will be used to perform some analysis of some sample manufacturing test data. The python script in this repository performs the necessary steps to pull down data from a Google Sheet and populate a SQLite database using a pandas dataframe as an intermediary. For convenience, this jupyter notebook starts from a csv file instead of from a Google Sheet. 

The steps are:
1. Read the data from the csv file into a pandas dataframe.
2. Clean up the data.
3. Plot and analyze the data.

Objective: Use the data to determine whether or not a test phase can be eliminated. If a test phase should be kept, explain why.

## Libraries

In [1]:
import unittest
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 1. Read data from csv

In [2]:
# Ingest the csv file into a pandas dataframe.
csv_file_name = "mfg-data.csv"
df = pd.read_csv(csv_file_name)

# Print the number of rows x columns in the dataframe.
print("dataframe shape:", df.shape)

# Check the first rows of the dataframe to see if the pandas import worked.
#df.head()

dataframe shape: (450, 32)


# 2. Cleanup the data

## Cleanup column names

In [3]:
# Cleanup column names to all lowercase with underscores instead of spaces.
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
#print("New column names:")
#df.head()

## Check key values for correctness

In [4]:
# Test the dataframe to make sure it contains correct values in some known positions.
class TestDataframe(unittest.TestCase):
	
	def test_some_column_names(self):
		self.assertEqual(df.columns.values.tolist()[0], "serial_number")
		self.assertEqual(df.columns.values.tolist()[31], "d_stddev_ol")

	def test_some_first_data_row_values(self):
		self.assertEqual(df.at[0,"serial_number"], "4A001H")
		self.assertEqual(df.at[0, "p_cl_2"], "29,335.58")

	def test_some_final_data_row_values(self):
		self.assertEqual(df.at[449, "serial_number"], "3C005V")
		self.assertEqual(df.at[449, "d_stddev_ol"], 0)
		
# Run the unittests
unittest.main(argv=[''], verbosity=1, exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


## Convert numerical data from strings to numbers

In [5]:
# First check what datatypes each column has.
# df.dtypes

In [6]:
# We can see that some of the numerical data is represented as floats and some as objects.
# It's better all the numerical data to floats.
columns_with_numerical_data = ["p_cl_2", "p_avg_cl_25", "p_avg_cl_8", "p_avg_ol", "p_stddev_ol",
                              "f_avg_cl_25", "f_avg_ol", "f_stddev_ol", "d_avg_cl_2", "d_avg_cl_25",
                              "d_avg_cl_8", "d_avg_ol"]

# Remove commas and whitespace
for column in columns_with_numerical_data:
    print(column)
    df[column] = df[column].apply(lambda x : x.strip().replace(',', '')) 
  
print("ok")
df[columns_with_numerical_data] = df[columns_with_numerical_data].astype(float)

p_cl_2
p_avg_cl_25
p_avg_cl_8
p_avg_ol
p_stddev_ol
f_avg_cl_25
f_avg_ol
f_stddev_ol
d_avg_cl_2
d_avg_cl_25
d_avg_cl_8
d_avg_ol
ok


## Convert categorical data from strings to numbers

In [7]:
# For example, plotting and analysis will be easier if we encode "Pass" as 1, "Fail" as 0, and "NaN" as -1
columns_with_categorical_data = ["test_result", "status_cl_2", "status_cl_25", "status_cl_8", "status_ol"]

status_mapping = {"Pass": 1, "Fail": 0}

for column in columns_with_categorical_data:
    print(column)
    df[column] = df[column].map(status_mapping)
    df[column] = df[column].fillna(-1)

df[columns_with_categorical_data] = df[columns_with_categorical_data].astype(int)

test_result
status_cl_2
status_cl_25
status_cl_8
status_ol


In [8]:
df.head()

,serial_number,date_created,testingsoftwareversion,test_result,status_cl_2,status_cl_25,status_cl_8,status_ol,p_cl_2,p_avg_cl_25,...,f_stddev_cl_8,f_stddev_ol,d_avg_cl_2,d_avg_cl_25,d_avg_cl_8,d_avg_ol,d_stddev_cl_2,d_stddev_cl_25,d_stddev_cl_8,d_stddev_ol
0,4A001H,1/23/2018 10:49,3.2,1,1,1,1,1,29335.58,29491.29,...,57.42083,45.50565,368.1233,4182.690,1350.583,2383.093,19.25788,17.21346,15.86988,11.74825
1,4A001H,1/23/2018 10:49,3.2,0,1,0,1,1,29283.49,29633.68,...,61.96784,44.54431,372.3033,4497.020,1339.677,2396.403,16.08281,985.31040,16.53243,11.21949
2,4A001P,1/23/2018 10:15,3.2,1,1,1,1,1,29305.18,29363.80,...,60.35593,47.61033,412.0700,3849.643,1316.220,2402.773,18.30341,16.72933,14.27912,11.31950
3,4A001M,1/23/2018 10:02,3.2,1,1,1,1,1,29281.21,29302.74,...,52.70877,52.20287,319.4800,3205.407,1072.523,2387.513,15.92340,30.93753,13.09100,12.09733
4,4A001K,1/23/2018 9:47,3.2,1,1,1,1,1,29390.77,29342.95,...,60.10137,41.39107,348.8233,3549.727,1196.373,2405.407,12.25349,18.39503,13.93871,10.74339


## Convert timestamps to pandas.timestamp

In [9]:
df["date_created"] = pd.to_datetime(df["date_created"])    
df.head()

,serial_number,date_created,testingsoftwareversion,test_result,status_cl_2,status_cl_25,status_cl_8,status_ol,p_cl_2,p_avg_cl_25,...,f_stddev_cl_8,f_stddev_ol,d_avg_cl_2,d_avg_cl_25,d_avg_cl_8,d_avg_ol,d_stddev_cl_2,d_stddev_cl_25,d_stddev_cl_8,d_stddev_ol
0,4A001H,2018-01-23 10:49:00,3.2,1,1,1,1,1,29335.58,29491.29,...,57.42083,45.50565,368.1233,4182.690,1350.583,2383.093,19.25788,17.21346,15.86988,11.74825
1,4A001H,2018-01-23 10:49:00,3.2,0,1,0,1,1,29283.49,29633.68,...,61.96784,44.54431,372.3033,4497.020,1339.677,2396.403,16.08281,985.31040,16.53243,11.21949
2,4A001P,2018-01-23 10:15:00,3.2,1,1,1,1,1,29305.18,29363.80,...,60.35593,47.61033,412.0700,3849.643,1316.220,2402.773,18.30341,16.72933,14.27912,11.31950
3,4A001M,2018-01-23 10:02:00,3.2,1,1,1,1,1,29281.21,29302.74,...,52.70877,52.20287,319.4800,3205.407,1072.523,2387.513,15.92340,30.93753,13.09100,12.09733
4,4A001K,2018-01-23 09:47:00,3.2,1,1,1,1,1,29390.77,29342.95,...,60.10137,41.39107,348.8233,3549.727,1196.373,2405.407,12.25349,18.39503,13.93871,10.74339


# 3. Data Analysis

## For each column, how many missing or strange values are there?

In [10]:
# TODO

## Are the different tests correlated?

In [11]:
# TODO
# Correlation heatmap of passes and fails for the 5 test categories
# If a certain test is not highly correlated with overall test success,
# the test might be a good candidate for removal.

## What variables are amiss when a test fails?

In [12]:
# TODO
# For a certain serial_number, plot how some process variables p, d, f change over time.

# Print out the last rows
df.tail()

,serial_number,date_created,testingsoftwareversion,test_result,status_cl_2,status_cl_25,status_cl_8,status_ol,p_cl_2,p_avg_cl_25,...,f_stddev_cl_8,f_stddev_ol,d_avg_cl_2,d_avg_cl_25,d_avg_cl_8,d_avg_ol,d_stddev_cl_2,d_stddev_cl_25,d_stddev_cl_8,d_stddev_ol
445,3H0008,2017-10-24 09:51:00,3.2,0,0,1,1,1,29285.51,29361.24,...,52.47752,35.79988,611.0733,6033.52,1996.680,2442.743,23.954190,389.47310,21.40199,11.049810
446,3H0008,2017-10-24 09:51:00,3.2,0,0,1,0,1,29363.19,29356.98,...,63.70134,35.45084,1271.8730,6684.52,2667.273,2457.003,22.603810,21.03237,19.64738,10.966720
447,3H0008,2017-10-24 09:51:00,3.2,0,0,0,0,0,28861.63,29190.44,...,141.22730,17.84553,4836.3500,8401.35,5024.970,5044.630,2.437761,91.98053,40.16275,14.810630
448,3C005V,2017-10-24 09:51:00,3.1,0,1,1,0,1,29418.69,28589.91,...,442.95470,15.84445,2740.2330,5402.94,3878.843,3555.233,14.347770,24.72053,199.14850,2.452964
449,3C005V,2017-10-24 09:51:00,3.1,0,1,1,0,-1,29345.65,28828.88,...,64.46614,0.00000,2825.4770,4936.55,3650.173,0.000,17.848800,13.27551,20.92475,0.000000
